In [68]:
import pandas as pd
from matplotlib import pyplot as plt
import requests
from requests.auth import HTTPBasicAuth
import time
%matplotlib inline

In [4]:
NUM_ITERATIONS = 10
def avg(l):
    return float(sum(l))/(len(l))

In [60]:
query0_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?vav
WHERE {
    ?vav rdf:type brick:VAV .
}
"""

query0_hod = """
SELECT ?vav
WHERE {
    ?vav rdf:type brick:VAV .
};
"""

query1_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT DISTINCT ?sensor ?room
WHERE {

    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    {?sensor bf:isPointOf ?vav }
    UNION
    {?sensor bf:isPointOf ?room }
}
"""
query1_hod = """
SELECT ?sensor ?room
WHERE {
    ?sensor rdf:type/rdfs:subClassOf* brick:Zone_Temperature_Sensor .
    ?room rdf:type brick:Room .
    ?vav rdf:type brick:VAV .
    ?zone rdf:type brick:HVAC_Zone .

    ?vav bf:feeds+ ?zone .
    ?zone bf:hasPart ?room .

    { ?sensor bf:isPointOf ?vav .
    OR
    ?sensor bf:isPointOf ?room . }
};
"""
query2_sparql = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX brick: <http://buildsys.org/ontologies/Brick#> 
PREFIX bf: <http://buildsys.org/ontologies/BrickFrame#> 
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { ?vlv_cmd rdf:type brick:Reheat_Valve_Command }
      UNION
      { ?vlv_cmd rdf:type brick:Cooling_Valve_Command }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
}
"""
query2_hod = """
SELECT ?vlv_cmd ?vav
WHERE {
    {
      { 
      ?vlv_cmd rdf:type brick:Reheat_Valve_Command .
      OR
      ?vlv_cmd rdf:type brick:Cooling_Valve_Command .
      }
    }
    ?vav rdf:type brick:VAV .
    ?vav bf:hasPoint+ ?vlv_cmd .
};
"""


In [30]:
def run(func, show_first=False):
    query_times = []
    for i in range(NUM_ITERATIONS):
        t1 = time.time()*1000
        resp = func()
        t2 = time.time()*1000
        if resp is None: break
        if i == 0 and show_first: print resp.content
        query_times.append(t2-t1)
        print i, '{0:.2f}'.format(t2-t1)
        time.sleep(.5)
    query_avg = avg(query_times)
    print "Took {0:.2f}ms".format(query_avg)

## Hod Benchmark

In [31]:
# Hod Configs
HOD_SERVER = "http://localhost:47808/query"

In [54]:
# run hod
!cd hod; ./runserver.sh ; cd ..
#!cd hod; docker kill hod ; docker rm hod; docker run -d --name hod -p47808:47808 gtfierro/hod

+ docker pull gtfierro/hod
Using default tag: latest
latest: Pulling from gtfierro/hod
Digest: sha256:08d6d8a894838ee06f0b0dad781093b0ce827d172e89f4c3887b0ae0531bf3a7
Status: Image is up to date for gtfierro/hod:latest
+ docker kill hod
hod
+ docker rm hod
hod
+ docker run -d --name hod -p47808:47808 gtfierro/hod
74d59951473f7e09800d16c73cd00a180c06c44c23c5bc347b0d29e25f219478


In [61]:
## query 0
def hod_query_0():
    resp = requests.post(HOD_SERVER, data=query0_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp

run(hod_query_0)

0 22.41
1 13.44
2 11.15
3 10.24
4 10.31
5 10.86
6 11.92
7 10.25
8 10.89
9 12.18
Took 12.37ms


In [62]:
## query 1https://github.com/neo4j-contrib/sparql-plugin
def hod_query_1():
    resp = requests.post(HOD_SERVER, data=query1_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp

run(hod_query_1)

0 68.18
1 71.53
2 62.78
3 33.21
4 58.80
5 73.94
6 62.97
7 20.97
8 77.81
9 42.90
Took 57.31ms


In [63]:
## query 2
def hod_query_2():
    resp = requests.post(HOD_SERVER, data=query2_hod)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(hod_query_2)

0 79.47
1 70.32
2 59.21
3 61.16
4 70.08
5 60.61
6 79.32
7 58.99
8 61.97
9 52.35
Took 65.35ms


## Fuseki Benchmark

In [42]:
# Fuseki Configs
FUSEKI_SERVER = "http://localhost:3031/berkeley/query"

In [49]:
!cd fuseki; ./runserver.sh ; cd ..

+ docker pull gtfierro/fuseki
Using default tag: latest
latest: Pulling from gtfierro/fuseki
Digest: sha256:258ffe22f17a04e7792b8fedcb44f4b5b62df627958048f463d99ad26304b7d2
Status: Image is up to date for gtfierro/fuseki:latest
+ docker kill fuseki
fuseki
+ docker rm fuseki
fuseki
+ docker run -d --name fuseki -p3031:3030 gtfierro/fuseki
7424c18b00668d4f7558ad4e478728239c07d338f3d3025ce29a17bac3b47116


In [65]:
## query 0
def fuseki_query_0():
    resp = requests.post(FUSEKI_SERVER, params={'query':query0_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_0)

0 17.99
1 20.61
2 19.67
3 15.72
4 20.49
5 27.81
6 34.06
7 20.10
8 19.05
9 17.29
Took 21.28ms


In [76]:
## query 1
def fuseki_query_1():
    resp = requests.post(FUSEKI_SERVER, params={'query':query1_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_1)

KeyboardInterrupt: 

In [ ]:
## query 2
def fuseki_query_2():
    resp = requests.post(FUSEKI_SERVER, params={'query':query2_sparql})
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(fuseki_query_2, True)

## Alegrograph Benchmark

In [71]:
ALEGRO_SERVER = "http://localhost:10035/repositories/berkeley/sparql"
auth = HTTPBasicAuth('root','asdfasdf')  

In [72]:
def alegro_query_0():
    resp = requests.post(ALEGRO_SERVER, data={'query': query0_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_0)

0 35.25
1 26.94
2 12.15
3 11.64
4 19.64
5 11.99
6 11.60
7 19.51
8 17.60
9 9.99
Took 17.63ms


In [75]:
def alegro_query_1():
    resp = requests.post(ALEGRO_SERVER, data={'query': query1_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_1)

0 1858.43
1 1853.82
2 1796.68
3 1814.48
4 1881.41
5 1928.43
6 1924.16
7 1804.17
8 1815.61
9 1904.54
Took 1858.17ms


In [74]:
def alegro_query_2():
    resp = requests.post(ALEGRO_SERVER, data={'query': query2_sparql}, auth=auth)
    if not resp.ok:
        print resp, resp.reason
        return None
    return resp
run(alegro_query_2)

0 870.97
1 833.52
2 831.48
3 866.88
4 837.25
5 840.01
6 863.29
7 841.03
8 838.78
9 833.15
Took 845.64ms
